In [1]:
## header file function (need to be script )
import sys
import numpy as np
import vireoSNP
from vireoSNP.utils.vireo_base import match
from vireoSNP.utils.vcf_utils import load_VCF, write_VCF, parse_donor_GPb
from vireoSNP.utils.vcf_utils import read_sparse_GeneINFO, GenoINFO_maker
import matplotlib.pyplot as plt
import pandas as pd
from vireoSNP.utils.io_utils import match_donor_VCF
import re as re
def VCF_to_sdf(bulk_data):
    df_empty=pd.DataFrame(columns=["variants","AD","DP","GT"])
    bulk_dat = read_sparse_GeneINFO(bulk_data['GenoINFO'], keys=['AD', 'DP'])
    df_empty["variants"]=bulk_data["variants"]
    df_empty["AD"]=bulk_dat["AD"].toarray()
    df_empty["DP"]=bulk_dat["DP"].toarray()
    return (df_empty)

def rematch_vcf(bulk_data,donor_vcf):
    df_bulk=VCF_to_sdf(bulk_data)
    df_donor=pd.DataFrame(columns=["variants","GT"])
    df_donor["variants"]=donor_vcf["variants"]
    df_donor["GT"]=donor_vcf["GenoINFO"]["GT"]     
    donor_tensor = vireoSNP.vcf.parse_donor_GPb(donor_vcf['GenoINFO']["GT"], "GT")
    itersect_variants=pd.Series(list(set(df_donor["variants"]).intersection(set(df_bulk["variants"]))))
    id_d=df_donor["variants"].isin(itersect_variants)
    id_b=df_bulk["variants"].isin(itersect_variants)
    df_bulk=df_bulk[id_b]
    df_donor=df_donor[id_d]
    donor_tensor=donor_tensor[id_d]
    return(df_bulk,df_donor,donor_tensor)
def rematch_vcf_annotated(bulk_data,donor_vcf,donor_vcf_anno):
    df_bulk=VCF_to_sdf(bulk_data)
    df_donora=pd.DataFrame(columns=["variants","function","genes","avsnp","GT"])
    df_donora["variants"]=donor_vcf_anno["variants"]  
    df_donora["function"]=donor_vcf_anno["function."]
    df_donora["genes"]=donor_vcf_anno["genes"]
    df_donora["avsnp"]=donor_vcf_anno["avsnp"]
    df_donor=pd.DataFrame(columns=["variants","GT"])
    df_donor["variants"]=donor_vcf["variants"]
    df_donor["GT"]=donor_vcf["GenoINFO"]["GT"]     
    donor_tensor = vireoSNP.vcf.parse_donor_GPb(donor_vcf['GenoINFO']["GT"], "GT")
    itersect_variants=pd.Series(list(set(df_donora["variants"]).intersection(set(df_bulk["variants"]))))
    id_d=df_donor["variants"].isin(itersect_variants)
    id_b=df_bulk["variants"].isin(itersect_variants)
    id_da=df_donora["variants"].isin(itersect_variants)
    df_bulk=df_bulk[id_b]
    df_donor=df_donor[id_d]
    df_donora=df_donora[id_da]
    donor_tensor=donor_tensor[id_d]
    df_donora["GT"]=df_donora["GT"]=df_donor["GT"].tolist()
    return(df_bulk,df_donora,donor_tensor)
def merged_VCF_to_sdf(bulk_data_merge):
    df_empty=pd.DataFrame(columns=["variants","AD","DP","OTH"])
    df_empty["variants"]=bulk_data_merge["variants"]
    testdf=pd.DataFrame(np.array(bulk_merged_data['FixedINFO']["INFO"]))
    df_empty["AD"]=testdf[0].str.split(';',expand=True)[0].str.split('=',expand=True)[1].astype(float)
    df_empty["DP"]=testdf[0].str.split(';',expand=True)[1].str.split('=',expand=True)[1].astype(float)
    df_empty["OTH"]=testdf[0].str.split(';',expand=True)[2].str.split('=',expand=True)[1].astype(float)

    

    return(df_empty)
def rematch_merge_vcf_annotated(bulk_data,donor_vcf,donor_vcf_anno):
    df_bulk=merged_VCF_to_sdf(bulk_data)
    df_donora=pd.DataFrame(columns=["variants","function","genes","avsnp","GT"])
    df_donora["variants"]=donor_vcf_anno["variants"]  
    df_donora["function"]=donor_vcf_anno["function."]
    df_donora["genes"]=donor_vcf_anno["genes"]
    df_donora["avsnp"]=donor_vcf_anno["avsnp"]
    df_donor=pd.DataFrame(columns=["variants","GT"])
    df_donor["variants"]=donor_vcf["variants"]
    df_donor["GT"]=donor_vcf["GenoINFO"]["GT"]     
    donor_tensor = vireoSNP.vcf.parse_donor_GPb(donor_vcf['GenoINFO']["GT"], "GT")
    itersect_variants=pd.Series(list(set(df_donora["variants"]).intersection(set(df_bulk["variants"]))))
    id_d=df_donor["variants"].isin(itersect_variants)
    id_b=df_bulk["variants"].isin(itersect_variants)
    id_da=df_donora["variants"].isin(itersect_variants)
    df_bulk=df_bulk[id_b]
    df_donor=df_donor[id_d]
    df_donora=df_donora[id_da]
    donor_tensor=donor_tensor[id_d]
    df_donora["GT"]=df_donora["GT"]=df_donor["GT"].tolist()
    return(df_bulk,df_donora,donor_tensor)
def find_all(data, s):
    r_list = []
    for r in range(len(data)):
        r_list.append( s in data[r])
 
    return(r_list)
def removedfault(df_bulk,df_donora,donor_tensor):
    id_d=[]
    for bool in find_all(np.array(df_donora["GT"]).tolist(),"./."):
        id_d.append(not bool)
    df_donor=df_donora[id_d]
    df_gt=donor_tensor[id_d]
    id_b=df_bulk["variants"].isin(df_donor["variants"])
    df_b=df_bulk[id_b]
    return(df_b,df_donor,df_gt)

In [2]:
def rematch_merge_vcf_noannotated(bulk_data,donor_vcf):
    df_bulk=merged_VCF_to_sdf(bulk_data)
    df_donor=pd.DataFrame(columns=["variants","GT"])
    df_donor["variants"]=donor_vcf["variants"]
    df_donor["GT"]=donor_vcf["GenoINFO"]["GT"]
    donor_tensor = vireoSNP.vcf.parse_donor_GPb(donor_vcf['GenoINFO']["GT"], "GT")
    itersect_variants=pd.Series(list(set(df_donor["variants"]).intersection(set(df_bulk["variants"]))))
    id_d=df_donor["variants"].isin(itersect_variants)
    id_b=df_bulk["variants"].isin(itersect_variants)
    df_bulk=df_bulk[id_b]
    df_donor=df_donor[id_d]
    donor_tensor=donor_tensor[id_d]
    return(df_bulk,df_donor,donor_tensor)

In [3]:
def prediction_bulk(df_b,df_d,df_gt,donor_vcf,pre_model,genelist):
    gene_unique=genelist
    new=pd.DataFrame(index=gene_unique,columns=donor_vcf['samples']+["chi","p","numSNP"])
    pre_psi=pre_model
    test_d=df_d
    test_b=df_b
    test_gt=df_gt
    sample_list=donor_vcf['samples']
    nk=len(donor_vcf['samples'])
    for i in range(len(gene_unique)):
        sub_d=test_d[test_d["genes"]==gene_unique[i]]
        sub_b=test_b[test_b["variants"].isin(test_d[test_d["genes"]==gene_unique[i]]["variants"])]
        sub_gt=test_gt[test_d["genes"]==gene_unique[i]]
        a=np.array(sub_b["AD"])
        d=np.array(sub_b["DP"])
        a=np.array(list(map(float,a)))
        d=np.array(list(map(float,d)))
        model =vireoSNP.VireoBulk(nk)
        model.fit(a,d,sub_gt,learn_theta=False )
        pv=vireoSNP.LikRatio_test(model.psi,pre_psi,a,d,sub_gt,model.theta)
        for j in range(len(model.psi)):
            new[sample_list[j]][i]=model.psi[j]
        new["chi"][i]=pv[0]
        new["p"][i]=pv[1]
        new["numSNP"][i]=len(a)
    return(new)

In [4]:
donor_vcf=load_VCF(   "/home/flyflyzhizhi/Vireobulk_analysis/data/filter_pbmc10donors.vcf.gz",    biallelic_only=True,    load_sample=True, sparse=False , format_list=None)
####count based model
##bulk_merged_data=load_VCF(   "/media/wgs/subset_stimulation_bam/pbmc10donor_bulk/cellSNP.cells.vcf.gz",    biallelic_only=True,    load_sample=True,    format_list=None,)
#### UMI based model
bulk_merged_data=load_VCF("/home/flyflyzhizhi/Vireobulk_analysis/data/cellSNP.base.vcf.gz", biallelic_only=True,load_sample=False,format_list=None,)
###processedcsv
df_anno=pd.read_csv("/home/flyflyzhizhi/Vireobulk_analysis/data/scvcf_annotated_processed.csv")

In [5]:
len(donor_vcf['samples'])

10

In [ ]:
%%time
df_b=merged_VCF_to_sdf(bulk_merged_data)

In [ ]:
df_d=merged_VCF_to_sdf(donor_vcf)

In [ ]:
bulk

In [ ]:
df_b,df_d,m_GT=rematch_vcf(bulk_merged_data, donor_vcf)

In [ ]:
#count_bulk_data
count_bulk_data=load_VCF(   "/media/wgs/subset_stimulation_bam/pbmc10donor_bulk/cellSNP.cells.vcf.gz",    biallelic_only=True,    load_sample=True,    format_list=None,)
cf_b,cf_d,cm_GT=rematch_merge_vcf_annotated(count_bulk_data,donor_vcf,df_anno)
cf_b,cf_d,cm_GT=removedfault(cf_b,cf_d,cm_GT)

In [ ]:
donor_tensor = vireoSNP.vcf.parse_donor_GPb(donor_vcf['GenoINFO']["GT"], "GT")

In [ ]:
donor_tensor

In [ ]:
VCF_to_sdf(bulk_merged_data,)

In [6]:
%%time
df_b,df_d,m_GT=rematch_merge_vcf_annotated(bulk_merged_data,donor_vcf,df_anno)

CPU times: user 2.28 s, sys: 729 ms, total: 3.01 s
Wall time: 3.01 s


In [ ]:
%%time 
dft_b,dft_d,mtt_GT=rematch_merge_vcf_noannotated(bulk_merged_data,donor_vcf)

In [ ]:
dft_d

In [ ]:
df_d

In [ ]:
dft_b,dft_d,mtt_GT=removedfault(dft_b,dft_d,mtt_GT)

In [7]:
df_b,df_d,m_GT=removedfault(df_b,df_d,m_GT)

In [ ]:
dft_d

In [8]:
df_d

,variants,function,genes,avsnp,GT
0,chr1_14574_A_G,ncRNA_exonic,WASH7P,rs28503599,"[0/0, 0/1, 0/1, 0/0, 0/0, 0/0, 0/1, 0/0, 0/1, ..."
1,chr1_14590_G_A,ncRNA_exonic,WASH7P,rs707679,"[0/0, 0/1, 0/1, 0/0, 0/0, 0/1, 0/1, 0/0, 0/0, ..."
2,chr1_14599_T_A,ncRNA_exonic,WASH7P,rs707680,"[0/1, 0/1, 0/1, 0/0, 0/0, 0/1, 0/1, 0/1, 0/1, ..."
3,chr1_14604_A_G,ncRNA_exonic,WASH7P,rs541940975,"[0/1, 0/1, 0/1, 0/0, 0/0, 0/1, 0/1, 0/1, 0/1, ..."
4,chr1_14610_T_C,ncRNA_exonic,WASH7P,rs878986575,"[0/1, 0/1, 0/1, 0/0, 0/0, 0/1, 0/1, 0/1, 0/1, ..."
...,...,...,...,...,...
128180,chrX_155999818_C_T,intronic,IL9R,rs3093470,"[1/1, 0/0, 1/1, 1/1, 0/1, 0/1, 0/1, 1/1, 1/1, ..."
128181,chrX_155999869_A_G,intronic,IL9R,rs3091261,"[1/1, 1/1, 1/1, 1/1, 1/1, 1/1, 1/1, 1/1, 1/1, ..."
128182,chrX_155999957_G_C,intronic,IL9R,rs3093472,"[1/1, 0/0, 1/1, 1/1, 0/1, 0/1, 0/1, 1/1, 1/1, ..."
128183,chrX_156025297_G_A,downstream,DDX11L16,rs185932868,"[0/1, 0/0, 0/1, 0/0, 0/1, 0/0, 0/0, 0/1, 0/1, ..."


In [9]:
m_GT.shape

(127155, 10, 3)

In [10]:
gene_unique=np.unique(df_d["genes"].tolist())

In [11]:
a=np.array(df_b["AD"])
d=np.array(df_b["DP"])
a=np.array(list(map(float,a)))
d=np.array(list(map(float,d)))

In [12]:
model =vireoSNP.VireoBulk(10)

In [13]:
%%time
model.fit(a,d,m_GT,learn_theta=True )

CPU times: user 5min 59s, sys: 5.71 s, total: 6min 5s
Wall time: 23 s


In [14]:
model.theta

array([0.01338161, 0.44676019, 0.99255214])

In [15]:
#test
model.psi

array([0.05361776, 0.03054629, 0.05080027, 0.08600668, 0.14832878,
       0.07819137, 0.05603827, 0.12345294, 0.29600931, 0.07700833])

In [43]:
model.psi

array([0.05361776, 0.03054629, 0.05080027, 0.08600668, 0.14832878,
       0.07819137, 0.05603827, 0.12345294, 0.29600931, 0.07700833])

In [19]:
    model.n_GT

3

In [46]:
type(model.psi)

numpy.ndarray

In [47]:
str(model.psi).replace("\n","")

'[0.05361776 0.03054629 0.05080027 0.08600668 0.14832878 0.07819137 0.05603827 0.12345294 0.29600931 0.07700833]'

In [49]:
with open("modelsummary.txt","w") as f:
    f.write(','.join(donor_vcf['samples']))
    f.write("\n")
    f.write(str(model.psi).replace("\n",""))
    f.write("\n")
    f.write(str(model.theta))

In [ ]:
donor_vcf['samples']

In [ ]:
premodel=model.psi

In [ ]:
premodel

In [ ]:
%%time
bulk_demuti_result= prediction_bulk(df_b,df_d,m_GT,donor_vcf,premodel,gene_unique)

In [ ]:
bulk_demuti_result

In [ ]:
bulk_demuti_result.to_csv("/media/wgs/subset_stimulation_bam/pbmc10donor/annotated_expo_umi2.csv")

In [50]:
import os

In [53]:
os.path.join(os.getcwd(),"output.csv")

'/home/flyflyzhizhi/Vireobulk_analysis/pipeline/output.csv'